## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
f_app_train = os.path.join('./data/', 'application_train.csv')
f_app_test = os.path.join('./data/', 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [3]:
ages = pd.DataFrame({'age': app_train['DAYS_BIRTH'] / 365})
ages.describe()

,age
count,307511.000000
mean,43.936973
std,11.956133
min,20.517808
25%,34.008219
50%,43.150685
75%,53.923288
max,69.120548


### 等寬劃分

In [4]:
ages['equal_width_age'] = pd.cut(ages['age'], 4)
ages['equal_width_age'].value_counts()

(32.668, 44.819]    100808
(44.819, 56.97]      84685
(20.469, 32.668]     66536
(56.97, 69.121]      55482
Name: equal_width_age, dtype: int64

In [5]:
app_train['equal_width_age'] = pd.cut(app_train['AMT_GOODS_PRICE'], 4)
app_train['equal_width_age'].value_counts()

(36490.5, 1042875.0]      274655
(1042875.0, 2045250.0]     31580
(2045250.0, 3047625.0]       968
(3047625.0, 4050000.0]        30
Name: equal_width_age, dtype: int64

### 等頻劃分

In [6]:
ages['equal_freq_age'] = pd.qcut(ages['age'], 4)
ages['equal_freq_age'].value_counts()

(43.151, 53.923]    76887
(20.517, 34.008]    76884
(34.008, 43.151]    76877
(53.923, 69.121]    76863
Name: equal_freq_age, dtype: int64

In [7]:
app_train['equal_width_age'] = pd.qcut(app_train['AMT_GOODS_PRICE'], 4)
app_train['equal_width_age'].value_counts()

(40499.999, 238500.0]    79877
(238500.0, 450000.0]     78102
(679500.0, 4050000.0]    75534
(450000.0, 679500.0]     73720
Name: equal_width_age, dtype: int64